In [91]:
from pgmpy.models import BayesianNetwork, BayesianModel
from pgmpy.estimators import K2Score, BayesianEstimator
import pandas as pd
import random

def k2(dataset, parents_nmax):
    variables = list(dataset.columns)
    print(f'Ordem das variáveis iniciais: {variables}')
    random_order = random.sample(variables, len(variables))
    print(f'Ordem das variáveis aleatórias: {random_order}')
    estimator = K2Score(dataset)
    model = BayesianNetwork()
    nodes = list(random_order)  # nome dos nos
    model.add_nodes_from(nodes)

    for i in range(1, len(nodes)):
        node = nodes[i]  # no atual
        previous_nodes = nodes[:i]  # nos anteriores ao no atual
        parents = []  # pais do no atual
        P_old = estimator.local_score(node, parents)
        proceed = True

        while proceed and (len(parents) < parents_nmax):
            candidates = list(set(previous_nodes) - set(parents))  # nos candidatos para pais do no atual
            P_new = P_old  # new probability

            for candidate in candidates:
                candidate_score = estimator.local_score(node, parents + [candidate])  # pontuacao dos nos pais

                if candidate_score > P_new:
                    candidates_best = candidate  # melhor candidato para no pai
                    P_new = candidate_score

            if P_new > P_old:
                P_old = P_new
                parents.append(candidates_best)
                model.add_edge(candidates_best, node)

            else:
                proceed = False
    md = {'Estrutura do Modelo': list(model.edges)}
    score = {'Score': estimator.score(model)}
    return md, model, score

# Função para encontrar as CPDs
def tabular_cpd(model, data):
    for column in data.columns:
        data[column] = pd.Categorical(data[column])
    estimator = BayesianEstimator(model, data)
    cpds = [estimator.estimate_cpd(node) for node in model.nodes]
    return cpds

data = pd.read_csv('contact-lenses.csv')
best_model = k2(data, 4)
cpds = tabular_cpd(best_model[1], data)

print(f'Modelo: {best_model}')


Ordem das variáveis iniciais: ['age', 'spectacle-prescrip', 'astigmatism', 'tear-prod-rate', 'contact-lenses']
Ordem das variáveis aleatórias: ['spectacle-prescrip', 'contact-lenses', 'age', 'tear-prod-rate', 'astigmatism']
Modelo: ({'Estrutura do Modelo': [('contact-lenses', 'tear-prod-rate'), ('contact-lenses', 'astigmatism')]}, <pgmpy.models.BayesianNetwork.BayesianNetwork object at 0x000001A1BEAD94F0>, {'Score': -98.72503647411926})


In [92]:
from pgmpy.readwrite import XMLBIFWriter

# Especifique o caminho do arquivo onde deseja salvar o arquivo XMLBIF
file_path = "modelo.xmlbif"

# Adicione as CPDs ao modelo
for cpd in cpds:
    best_model[1].add_cpds(cpd)

# Escreva o modelo no formato XMLBIF
writer = XMLBIFWriter(best_model[1]).write_xmlbif(file_path)

print(f"O arquivo XMLBIF foi gerado com sucesso em: {file_path}")


O arquivo XMLBIF foi gerado com sucesso em: modelo.xmlbif


GERAÇÃO DE 10 MODELOS DE FORMA ALEATÓRIA

In [94]:
from pgmpy.readwrite import BIFReader
from pgmpy.models import BayesianNetwork, BayesianModel
from pgmpy.estimators import K2Score, BayesianEstimator
from pgmpy.sampling import BayesianModelSampling
import pandas as pd
import random

def k2(dataset, parents_nmax, num_iterations):
    variables = list(dataset.columns)
    print(f'Ordem das variáveis iniciais: {variables}')
    estimator = K2Score(dataset)
    models_and_scores = []

    for iteration in range(num_iterations):
        random_order = random.sample(variables, len(variables))
        print(f'Ordem das variáveis aleatórias (iteração {iteration+1}): {random_order}')
        model = BayesianNetwork()
        model.add_nodes_from(random_order)

        for i in range(1, len(random_order)):
            node = random_order[i]  # no atual
            previous_nodes = random_order[:i]  # nos anteriores ao no atual
            parents = []  # pais do no atual
            P_old = estimator.local_score(node, parents)
            proceed = True

            while proceed and (len(parents) < parents_nmax):
                candidates = list(set(previous_nodes) - set(parents))  # nos candidatos para pais do no atual
                P_new = P_old  # new probability

                for candidate in candidates:
                    candidate_score = estimator.local_score(node, parents + [candidate])  # pontuacao dos nos pais

                    if candidate_score > P_new:
                        candidates_best = candidate  # melhor candidato para no pai
                        P_new = candidate_score

                if P_new > P_old:
                    P_old = P_new
                    parents.append(candidates_best)
                    model.add_edge(candidates_best, node)

                else:
                    proceed = False

        score = estimator.score(model)
        models_and_scores.append((model, score))
        print(f"Iteração {iteration + 1} - Score: {score}")

    return models_and_scores

# Função para encontrar as CPDs
def tabular_cpd(model, data):
    for column in data.columns:
        data[column] = pd.Categorical(data[column])
    estimator = BayesianEstimator(model, data)
    cpds = [estimator.estimate_cpd(node) for node in model.nodes]
    return cpds

data = pd.read_csv('contact-lenses.csv')
num_iterations = 10
models_and_scores = k2(data, 4, num_iterations)

print("\nModelos e Scores:")
for iteration, (model, score) in enumerate(models_and_scores):
    print(f"Iteração {iteration + 1} - Modelo: {model.edges}, Score: {score}")


Ordem das variáveis iniciais: ['age', 'spectacle-prescrip', 'astigmatism', 'tear-prod-rate', 'contact-lenses']
Ordem das variáveis aleatórias (iteração 1): ['contact-lenses', 'astigmatism', 'tear-prod-rate', 'spectacle-prescrip', 'age']
Iteração 1 - Score: -98.72503647411928
Ordem das variáveis aleatórias (iteração 2): ['tear-prod-rate', 'astigmatism', 'contact-lenses', 'spectacle-prescrip', 'age']
Iteração 2 - Score: -100.67090227820763
Ordem das variáveis aleatórias (iteração 3): ['astigmatism', 'spectacle-prescrip', 'contact-lenses', 'tear-prod-rate', 'age']
Iteração 3 - Score: -99.00802002578952
Ordem das variáveis aleatórias (iteração 4): ['astigmatism', 'contact-lenses', 'spectacle-prescrip', 'tear-prod-rate', 'age']
Iteração 4 - Score: -99.00802002578952
Ordem das variáveis aleatórias (iteração 5): ['contact-lenses', 'age', 'spectacle-prescrip', 'tear-prod-rate', 'astigmatism']
Iteração 5 - Score: -98.72503647411929
Ordem das variáveis aleatórias (iteração 6): ['tear-prod-rate',